In [1]:
##############################################################################
# IMPORT ALL TIME PACKAGES
##############################################################################
import numpy as np
import pandas as pd

import pickle, time, random, sys, warnings
# sys.path is a list of absolute path strings
sys.path.append('/home/d/dlr10/Documents/02_Statitics_modelling/0_FunctionsScripts')
import Loading_Data_Functions as load_fn
import FineTuning_Functions as FineTuning
import Learning_Curves_Functions as LearningCurves

import warnings
warnings.filterwarnings('ignore')


In [2]:
# ##############
# 1. LOAD DATA ---------------------------------------------------------------
# ============================================================================
t = time.time()
path = r'/home/d/dlr10/Documents/02_Statitics_modelling/DataSets/'
df_patients_16_18, df_admissions_16_18, df_eobs_16_18 = load_fn.Load_data()
df_patients_19_20, df_admissions_19_20, df_eobs_19_20 = load_fn.Load_data('2019_2020')

X_data_16_18 = pickle.load(open(path + 'df_ts_2016_18.pickle','rb')).drop(columns = 'no_sample_series')
X_data_19_20 = pickle.load(open(path + 'df_ts_2019_20.pickle','rb')).drop(columns = 'no_sample_series')


In [3]:
X_data_16_18.head(2)

,admission_id,rr,ews,heart_rate,temperature,sbp,dbp,Oxygen_Saturation,Assisted_O2,Confusion,...,BASAB,ALB,ALP,BILI,sex,ethnicity,age_at_admin,Comorb_score,Spcfc_Comorb,Mortality
0,30454500,16.0,9.0,143.000000,36.4,81.000000,49.000000,90.000000,1.0,1.0,...,0.02,30.0,120.5,8.0,1.0,0.0,72.0,0.0,1.0,0.0
1,30454500,15.0,7.0,115.666667,36.5,88.666667,62.333333,92.666667,1.0,1.0,...,0.02,30.0,120.5,8.0,1.0,0.0,72.0,0.0,1.0,0.0


In [4]:
def create_df_fro_ews_modelling(df, len_ts, tar_samp, domn_var):
    list_admissions = df['admission_id'].unique().tolist()

    list_encodes = []
    df_final     = pd.DataFrame()
    for i, admission_id in enumerate(list_admissions):
        admission  = df[df['admission_id'] == admission_id].copy()
        ews_target = int(admission.iloc[tar_samp]['ews'])
        #adm_encode = ews_target*[0] + [1] + (domn_var-ews_target-1)*[0]
        #list_encodes.append(adm_encode)
        list_encodes.append(ews_target)
        df_final = pd.concat([df_final, admission.iloc[:len_ts]])

        #break
        #if i == 2:break
    df_final = df_final.drop(columns = ['ews'])
    return df_final, list_encodes

In [5]:
# ============================================================================
# CREATE DATA WITH ALL FEATURES AND 1 DAY OF TS,AND INITIAL SAMPLE OF EWS AT 3RD DAY
# ============================================================================

len_ts   = 48
tar_samp = 96
domn_var = 20 #domain target variable
t = time.time ()
X_data_16_18_ews, list_encodes_16_18 = create_df_fro_ews_modelling(X_data_16_18, len_ts, tar_samp, domn_var)
print("time elapsed getting set of all feature 1 dayto day 3", time.time() -t )

time elapsed getting set of all feature 1 dayto day 3 544.5152049064636


In [6]:
t = time.time()
X_data_19_20_ews, list_encodes_19_20 = create_df_fro_ews_modelling(X_data_19_20, len_ts, tar_samp, domn_var)
print("time elapsed getting set of all feature 1 dayto day 3", time.time() -t )

time elapsed getting set of all feature 1 dayto day 3 379.95883345603943


In [7]:
path = r'/home/d/dlr10/Documents/02_Statitics_modelling/DataSets/'
pickle.dump([X_data_16_18_ews, list_encodes_16_18],  open(path + 'df_ts_2016_18_ews_1d.pickle', 'wb'))
pickle.dump([X_data_19_20_ews, list_encodes_19_20],  open(path + 'df_ts_2019_20_ews_1d.pickle', 'wb'))

In [8]:
# ============================================================================
# CREATE DATA WITH ALL FEATURES AND 2 DAY OF TS,AND LAST SAMPLE OF EWS AT 3RD DAY
# ============================================================================
len_ts   = 48 * 2
tar_samp = 143
domn_var = 20 #domain target variable
t = time.time ()
X_data_16_18_ews, list_encodes_16_18 = create_df_fro_ews_modelling(X_data_16_18, len_ts, tar_samp, domn_var)
print("time elapsed getting set of all feature 2ND day to LATE 3RD DAY", time.time() -t )

time elapsed getting set of all feature 2ND day to LATE 3RD DAY 687.7772452831268


In [9]:
t = time.time()
X_data_19_20_ews, list_encodes_19_20 = create_df_fro_ews_modelling(X_data_19_20, len_ts, tar_samp, domn_var)
print("time elapsed getting set of all feature 2ND day to LATE 3RD DAY", time.time() -t )

time elapsed getting set of all feature 2ND day to LATE 3RD DAY 515.4572007656097


In [10]:
path = r'/home/d/dlr10/Documents/02_Statitics_modelling/DataSets/'
pickle.dump([X_data_16_18_ews, list_encodes_16_18],  open(path + 'df_ts_2016_18_ews_2d.pickle', 'wb'))
pickle.dump([X_data_19_20_ews, list_encodes_19_20],  open(path + 'df_ts_2019_20_ews_2d.pickle', 'wb'))